In [205]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import xlsxwriter
import datetime

На входе
1. Выгрузки GA (в формате CSV) - это выгрузки из GA сделанные из отчетов без семплинга, без дальнейшего переименования, 
    * пример корректного названия : '\__offer\__krovelnye-materialy_\_ - month\__number'
        * название промоветки в имени файла отделено обратными слешами в начале и конце имени \_

2. Файл справочник(формат xlsx) - __от корректности и однородности этого файла зависит весь процесс обработки__.
    * Переименовываем файл: info.xlsx
    * Каждая вкладка(эксель страница) в инфо файле должна иметь уникальный path промоветки, который вытаскивается из ссылки
    * __Одна вкладка строго соответствует одной промоветке__, без ссылок на другие листы и старых вкладок с такой же юрл
    * Если у нас есть две вкладки с одинаковым юрл (path промоветке), скрипт сработает некорректно
3. Кладем все файлы в отдельную папку, проверяем что нет дублей файлов
4. Прописываем полный путь до папки в переменную path
---
На выходе
* файл report.xlsx с 3 вкладками:
    1. result_before_merge
    2. result_after_merge
    3. result_after_merge_by_lm

----

## Заполняем путь к папке с файлами 
* путь до папки со всеми файлами, вначале и конце пути закрывающие слеши /full path to folder/

__Вид пути для Windows__
    *  'c:\\path\\folder_name\\'
__Вид пути для Mac OS__
    *  '/path/folder_name/'

### Указываем путь здесь

In [206]:
path = '/Users/peter/Desktop/promolines_november/'

---

In [207]:
def get_csv_files_name_and_path(path):
    '''
    Открываем папку и извлекаем название ветки из каждой выгрузки
    Если файлов получено меньше, чем было - проверяем дубли в папке
    Возвращает словарь название_промоветки : полный_путь_до_файла
    '''
    dict_csv = {}
    k=0
    for filename in glob.glob(os.path.join(path, '*.csv')):
        k+=1
        name_file = filename.split('_')[-2]
        #print(k,'      ',name_file)
        path_file = str(filename)
        dict_csv[name_file] = path_file
    print(f'\nПолучено {len(dict_csv)} файлов в формате csv\n')
    return dict_csv

### Проверить здесь, что импортировалось корректное количество файлов

In [208]:
dict_with_path = get_csv_files_name_and_path(path)


Получено 77 файлов в формате csv



---

In [209]:
#создаем глобал переменные с пустыми датафреймами, в них будем писать всю информацию
result_before_merge = pd.DataFrame()
result_after_merge = pd.DataFrame()
result_after_merge_by_lm = pd.DataFrame()

In [210]:
def create_result_files(dict_with_path=dict_with_path):
    '''
    Создаем пустые файлы для записи в них результатов
    result_total (result_before_merge) - тотал для агрегатов после джойна по артикулам с файлом info
    result_promo_other (result_after_merge) - тотал для агрегатов по каждому файлу выгрузки (без джойна)
    result_after_merge_by_lm - расширенный файл для результатов по каждому LM коду после мержа с инфо файлом
    '''
    #генерируем структуру датафреймов
    #читаем первый файл, берем колоноки, создаем структуру дф
    temp = pd.read_csv(list(dict_with_path.items())[0][-1])
    temp.columns = [i.lower().replace(' ','_') for i in temp]
    generate_dict_for_df_structure = {i:[] for i in temp.columns if ((i != 'сегмент') and (i!='идентификатор_продукта'))}
    generate_dict_for_df_structure['url_id'] = []
    
    global result_before_merge
    global result_after_merge
    global result_after_merge_by_lm
    
    #создаем пустые дф из словаря
    result_before_merge = pd.DataFrame(generate_dict_for_df_structure)
    result_after_merge = pd.DataFrame(generate_dict_for_df_structure)
    
    #добавляем дополнительное поле в словарь для записи lm code
    generate_dict_for_df_structure['id'] = []
    #создаем пустой датафрейм из словаря
    result_after_merge_by_lm = pd.DataFrame(generate_dict_for_df_structure)
      
    #добавляем датафреймы в словарь и сохраняем на диск
#     data_frames = {'result_before_merge':result_before_merge, 'result_after_merge':result_after_merge,
#                   'result_after_merge_by_lm':result_after_merge_by_lm}
#     for key, value in data_frames.items():
#         writer = pd.ExcelWriter(path+key+'.xlsx', engine='xlsxwriter')
#         (data_frames[key]).to_excel(writer, sheet_name='test', index = False)
#         writer.save()

In [211]:
create_result_files()

In [212]:
def reading_files_from_folder(key, dict_with_path=dict_with_path):
    '''
    Читает файлы по сформированному словарю
    '''
    df = pd.read_csv(dict_with_path[key])
    return df

In [213]:
def get_result_before_merge_total(dict_with_path = dict_with_path):
    '''
    Считает агрегаты по каждой промоветке до мержа,
    пишет в готовый файл
    groupby_dictionary - формирует значения для .agg в groupby
    '''
    #result = pd.read_excel(path+'result_before_merge.xlsx')
    global result_before_merge
    for key, value in dict_with_path.items():
        temp = pd.read_csv(value)
        temp.columns = [i.lower().replace(' ','_') for i in temp]
        groupby_dictionary = {i:'sum' for i in temp.columns if i not in ['идентификатор_продукта','сегмент']}
        temp = temp.groupby('сегмент', as_index=False).agg(groupby_dictionary)
        temp['url_id'] = key
        temp.drop(['сегмент'], axis=1, inplace=True)
        result_before_merge = pd.concat([temp, result_before_merge])
#     writer = pd.ExcelWriter(path+'result_before_merge.xlsx', engine='xlsxwriter')
#     result_before_merge.to_excel(writer, sheet_name='test', index = False)
#     writer.save()

In [214]:
get_result_before_merge_total()

In [215]:
def aggregation_and_merge(promo_df, temp, promo_name):
    
    #датафрейма для записи результатов после джойна
    #total = pd.read_excel(path+'result_after_merge.xlsx')
    #датафрейм для записи агрегатов по промоветкам без джойна
    
    global result_after_merge
    global result_after_merge_by_lm
    
    '''
    получает вкладку из справочника и соответствующей ей датафрейм, мержит, считает агрегаты и сохраняет в глобальные датафреймы
    promo_df -> выгрузка из GA
    temp -> соответствующая вкладка из инфо файла
    '''
    
    #приводит column names в стандартный вид
    promo_df.columns = [i.lower().replace(' ','_') for i in promo_df]
    #меняем тип данных для операций
    for i in promo_df.columns:
        if i not in ['идентификатор_продукта','сегмент']:
            promo_df[i] = promo_df[i].astype('float')
    
    #генерируем словарь название колонки:'sum'
    groupby_dictionary = {i:'sum' for i in promo_df.columns if i not in ['идентификатор_продукта','сегмент']}
    
    #агрегируем данные по каждому LM коду (исключаем возможность дублирования LM кодов в выгрузки) 
    promo_df = promo_df.groupby('идентификатор_продукта', as_index=False).agg(groupby_dictionary).\
                                                        rename(columns={'идентификатор_продукта':'id'})
    object_values_in_id = ['not set','notset','(not set)']
    promo_df = promo_df[~promo_df['id'].isin(object_values_in_id)]
    promo_df['id'] = promo_df['id'].astype('int64')
    
    temp.drop_duplicates(subset = 'id', keep = 'first', inplace = True)
    #мержим promo_df и temp
    test_new = temp.merge(promo_df, on='id', how='left').fillna(0)
    
    #считаем тотал по одной промоветке
    promo_df_total_sum = test_new.groupby('url_id', as_index=False).agg(groupby_dictionary)

    
    #пишем в глобальную переменную, датафрейм result_after_merge_by_lm
    result_after_merge_by_lm = pd.concat([test_new,result_after_merge_by_lm])
    #и сохраняем каждый файл отдельно
    #writer_1 = pd.ExcelWriter(path+'done_'+promo_name+'.xlsx', engine = 'xlsxwriter')
    #test_new.to_excel(writer_1, sheet_name = 'test', index=False)
    #writer_1.save()
    
    
    #пишем в глобальную переменную result_after_merge 
    result_after_merge = pd.concat([promo_df_total_sum,result_after_merge])
    #writer_2 = pd.ExcelWriter(path+'result_after_merge.xlsx', engine='xlsxwriter')
    #total.to_excel(writer_2, sheet_name='test', index = False)
    #writer_2.save()

In [216]:
def dump_result_df():
    '''
    после прохождения по всем вкладкам в функции reading_info_file записываем 3 датафрейма в один xlsx файл и сохраняем
    '''
    global result_before_merge
    global result_after_merge
    global result_after_merge_by_lm
    
    #идем по словарю и сохраняем результирующие датафреймы в один файл на разные вкладки с именем key
    data_frames = {'result_before_merge':result_before_merge, 'result_after_merge':result_after_merge,
                  'result_after_merge_by_lm':result_after_merge_by_lm}
    current_month = (datetime.datetime.now()).month
    writer = pd.ExcelWriter(path+'report_month_'+str(current_month)+'.xlsx', engine='xlsxwriter')  
    for key, value in data_frames.items():
        data_frames[key] = data_frames[key].fillna(0)
        (data_frames[key]).to_excel(writer, sheet_name=key, index = False)
    writer.save()
    

In [217]:
def reading_info_file():
    '''
    Обрабатывает файл справочник
    Идет по названиям листов, сохраняет лист в дф,
    берет значение юрл со страницы, выделяет ключ и пробует передать его в ф-ю reading_files_from_folder
    '''
    #флаг на ошибки

    temp_path = path+'info.xlsx'
    xls = pd.ExcelFile(temp_path)
    list_of_sheets = [i for i in xls.sheet_names]
    #print(list_of_sheets)
    for i in list_of_sheets:
        error1 = False
        error2 = False
        
        #чтение датафрейма с конкретным листом
        df = pd.read_excel(xls, sheet_name=i, header=None).iloc[:,:2]
        #пробуем п
        try:
            df.columns = ['id','url_id']
        except Exception as e:
            print(f' {i}:Ошибка на этапе выделения ключа, проверить что на вкладке есть информация\n')
            error1 = True
        
        if error1 == False:
            
            if df.iloc[0,1][-1] == '/':
                key = df.iloc[0,1].split('/')[-2]
                df['url_id'] = key
            else:
                key = df.iloc[0,1].split('/')[-1]
            try:
                promo_df = reading_files_from_folder(key, dict_with_path=dict_with_path)
                #display(promo_df.head())
            
            except Exception as e:
                print(f'Вкладка {[i]} из info файла пропущена, не найдена соответствующая выгрузка')
                error2 = True
            
            if error2 == False:
                aggregation_and_merge(promo_df=promo_df, temp = df, promo_name=key)
    
    dump_result_df()

In [218]:
reading_info_file()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Вкладка ['заборы калитки ворота '] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['инструменты для дачи'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['мототехника для дачи'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['Смесители дизайнерский стиль'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['Смесители природный стиль'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['Смесители стиль шарм'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['товары для безопасн нового года'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['новогодн уличн гирлянды'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['гирлянды для дома'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['подарки для мужчин'] из info файла пропущена, не найдена соответствующая выгрузка
Вкладка ['необычные подарки'] из inf